In [ ]:
import pandas as pd
import os
import pyarrow as pa
import pyarrow.parquet as pq

In [2]:
merged_df = pd.read_csv("../data/processed/merged_df.csv")
merged_df.head()

/var/folders/r3/fhz0cy092rq1vdp607ycp5zc0000gn/T/ipykernel_21570/2222649024.py:1: DtypeWarning: Columns (5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  merged_df = pd.read_csv("../data/processed/merged_df.csv")


,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,6532F46EF12DCA8C,electric_bike,2025-06-12 15:08:06.700,2025-06-12 15:11:20.652,Mercer St & Bleecker St,5679.05,E 1 St & Bowery,5636.13,40.727064,-73.996621,40.724861,-73.992131,member
1,DE5D64BE54489439,electric_bike,2025-06-12 13:05:20.309,2025-06-12 13:22:18.812,45 Rd & 11 St,6319.01,43 Ave & 42 St,6251.06,40.747086,-73.949772,40.745347,-73.921942,member
2,A19F721536473DA2,classic_bike,2025-06-13 18:03:51.268,2025-06-13 18:11:24.704,95 St & 35 Ave,6465.04,84 St & Roosevelt Ave,6323.04,40.753010,-73.872660,40.747970,-73.882040,member
3,5695831104F572BB,classic_bike,2025-06-11 17:01:40.807,2025-06-11 17:24:19.780,4 Ave & 17 St,3739.06,Willoughby St & Ashland Pl,4587.02,40.665507,-73.993037,40.691780,-73.978770,member
4,08F1301EED21DFC7,electric_bike,2025-06-06 18:43:26.907,2025-06-06 18:48:48.209,Mercer St & Bleecker St,5679.05,St Marks Pl & 2 Ave,5669.10,40.727064,-73.996621,40.728419,-73.987140,member


In [3]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9084898 entries, 0 to 9084897
Data columns (total 13 columns):
 #   Column              Dtype  
---  ------              -----  
 0   ride_id             object 
 1   rideable_type       object 
 2   started_at          object 
 3   ended_at            object 
 4   start_station_name  object 
 5   start_station_id    object 
 6   end_station_name    object 
 7   end_station_id      object 
 8   start_lat           float64
 9   start_lng           float64
 10  end_lat             float64
 11  end_lng             float64
 12  member_casual       object 
dtypes: float64(4), object(9)
memory usage: 901.1+ MB


In [4]:
merged_df.describe()

,start_lat,start_lng,end_lat,end_lng
count,9.081215e+06,9.081215e+06,9.055271e+06,9.055271e+06
mean,4.073679e+01,-7.397155e+01,4.073661e+01,-7.397164e+01
std,4.073091e-02,2.892372e-02,4.067138e-02,2.894767e-02
min,4.063338e+01,-7.402682e+01,4.063338e+01,-7.408670e+01
25%,4.071222e+01,-7.399209e+01,4.071222e+01,-7.399213e+01
50%,4.073535e+01,-7.397898e+01,4.073524e+01,-7.397901e+01
75%,4.076030e+01,-7.395487e+01,4.076030e+01,-7.395493e+01
max,4.088630e+01,-7.384672e+01,4.088630e+01,-7.384672e+01


In [5]:
merged_df.isna().any()

ride_id               False
rideable_type         False
started_at            False
ended_at              False
start_station_name     True
start_station_id       True
end_station_name       True
end_station_id         True
start_lat              True
start_lng              True
end_lat                True
end_lng                True
member_casual         False
dtype: bool

In [6]:
merged_df.isna().sum()

ride_id                   0
rideable_type             0
started_at                0
ended_at                  0
start_station_name     3683
start_station_id       3683
end_station_name      28616
end_station_id        29669
start_lat              3683
start_lng              3683
end_lat               29627
end_lng               29627
member_casual             0
dtype: int64

In [7]:
cleaned_df = merged_df.fillna("Unknown")
cleaned_df.isna().any()

ride_id               False
rideable_type         False
started_at            False
ended_at              False
start_station_name    False
start_station_id      False
end_station_name      False
end_station_id        False
start_lat             False
start_lng             False
end_lat               False
end_lng               False
member_casual         False
dtype: bool

In [10]:
cleaned_df.head(5)

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,6532F46EF12DCA8C,electric_bike,2025-06-12 15:08:06.700,2025-06-12 15:11:20.652,Mercer St & Bleecker St,5679.05,E 1 St & Bowery,5636.13,40.727064,-73.996621,40.724861,-73.992131,member
1,DE5D64BE54489439,electric_bike,2025-06-12 13:05:20.309,2025-06-12 13:22:18.812,45 Rd & 11 St,6319.01,43 Ave & 42 St,6251.06,40.747086,-73.949772,40.745347,-73.921942,member
2,A19F721536473DA2,classic_bike,2025-06-13 18:03:51.268,2025-06-13 18:11:24.704,95 St & 35 Ave,6465.04,84 St & Roosevelt Ave,6323.04,40.75301,-73.87266,40.74797,-73.88204,member
3,5695831104F572BB,classic_bike,2025-06-11 17:01:40.807,2025-06-11 17:24:19.780,4 Ave & 17 St,3739.06,Willoughby St & Ashland Pl,4587.02,40.665507,-73.993037,40.69178,-73.97877,member
4,08F1301EED21DFC7,electric_bike,2025-06-06 18:43:26.907,2025-06-06 18:48:48.209,Mercer St & Bleecker St,5679.05,St Marks Pl & 2 Ave,5669.10,40.727064,-73.996621,40.728419,-73.98714,member


In [11]:
cleaned_df["started_at"] = pd.to_datetime(cleaned_df["started_at"])
cleaned_df["ended_at"] = pd.to_datetime(cleaned_df["ended_at"])

In [12]:
cleaned_df["started_at"] = cleaned_df["started_at"].dt.tz_localize("UTC")
cleaned_df["ended_at"] = cleaned_df["ended_at"].dt.tz_localize("UTC")

In [14]:
cleaned_df["member_casual"] = cleaned_df["member_casual"].astype("category")

In [15]:
cleaned_df["rideable_type"] = cleaned_df["rideable_type"].astype("category")

In [12]:
cleaned_df.duplicated().sum()

np.int64(0)

In [26]:
str_cols = ["start_station_id", "end_station_id", "start_station_name", "end_station_name", "ride_id"]
for col in str_cols:
    cleaned_df[col] = cleaned_df[col].astype("string")

In [ ]:
# List of your coordinate columns
float_cols = ["start_lat", "start_lng", "end_lat", "end_lng"]

for col in float_cols:
    cleaned_df[col] = pd.to_numeric(cleaned_df[col], errors="coerce").astype("float64")

In [27]:
cleaned_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9084898 entries, 0 to 9084897
Data columns (total 13 columns):
 #   Column              Dtype              
---  ------              -----              
 0   ride_id             string             
 1   rideable_type       category           
 2   started_at          datetime64[ns, UTC]
 3   ended_at            datetime64[ns, UTC]
 4   start_station_name  string             
 5   start_station_id    string             
 6   end_station_name    string             
 7   end_station_id      string             
 8   start_lat           float64            
 9   start_lng           float64            
 10  end_lat             float64            
 11  end_lng             float64            
 12  member_casual       category           
dtypes: category(2), datetime64[ns, UTC](2), float64(4), string(5)
memory usage: 779.8 MB


In [28]:
output_dir = os.path.join("..", "data", "processed") 
if not os.path.exists(output_dir): 
    os.mkdirs(output_dir, exist_ok=True) 

output_path = os.path.join(output_dir, "cleaned_df.parquet") 
    
cleaned_df.to_parquet(output_path, engine="pyarrow", index=False) 
print(f"File saved to {output_path}")

File saved to ../data/processed/cleaned_df.parquet
